In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from src import algorithms, Data_analysis_tools as Tools, stats_scraper2_0
import joblib
from keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

In [24]:
kwargs = {'Season': '2021-22'}

In [25]:
data = stats_scraper2_0.web_scraper2_0('teams_boxscore_trad_2022_first_half.csv',training_dataset=True, **kwargs)
data = algorithms.pre_process_cols_new(data)
data = algorithms.hollinger_formula(data)
data = algorithms.concat_home_away_stats(data)
data = algorithms.get_dummies(data)
data = algorithms.feature_eng(data)
data = algorithms.clean_data(data)
data = data.sort_values(by='DATE_Home', key=pd.to_datetime, ascending=False)
data.to_csv('validation_data_first_half_2021.csv')
data

Fetching Data
Done


,TEAM_Home,DATE_Home,W/L_Home,MIN_Home,FGM_Home,FGA_Home,FG%_Home,3PM_Home,3PA_Home,3P%_Home,...,OREB%_Home,OREB%_Away,STL%_Home,STL%_Away,TOV%_Home,TOV%_Away,FF_Home,FF_Away,GS_Home,GS_Away
2459,PHI,2022-04-10T00:00:00,W,24.0,22,42,0.524,3,12,0.250,...,31.578947,30.434783,16.075103,4.297305,9.177680,13.596193,7.253993,7.950363,56.2,43.6
2444,HOU,2022-04-10T00:00:00,L,24.0,21,47,0.447,10,27,0.370,...,19.230769,31.250000,6.440643,2.132815,3.714710,9.342302,4.719316,8.126647,45.9,67.8
2430,WAS,2022-04-10T00:00:00,L,24.0,22,44,0.500,4,14,0.286,...,16.666667,36.363636,13.483111,4.190131,7.446016,19.329897,5.967168,10.938491,44.7,43.1
2431,BKN,2022-04-10T00:00:00,W,24.0,29,42,0.690,3,9,0.333,...,46.153846,24.137931,7.188866,15.582149,16.812374,7.697044,10.356083,5.301831,71.7,45.2
2432,MIL,2022-04-10T00:00:00,L,24.0,17,47,0.362,5,16,0.313,...,14.285714,31.818182,5.941065,7.738980,10.373444,5.033557,6.947585,8.054054,28.8,77.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,HOU,2021-10-20T00:00:00,L,24.0,18,42,0.429,6,18,0.333,...,15.789474,50.000000,7.988241,22.417504,25.527570,12.468828,9.961059,9.325757,20.5,72.1
2,GSW,2021-10-19T00:00:00,W,24.0,20,49,0.408,4,16,0.250,...,14.285714,9.090909,10.035324,8.623068,17.080745,15.556938,9.645186,9.628859,35.9,38.5
1,BKN,2021-10-19T00:00:00,L,24.0,22,41,0.537,11,19,0.579,...,10.000000,26.666667,1.843005,8.393768,15.491867,4.649721,9.273017,6.954930,44.5,52.6
3,MIL,2021-10-19T00:00:00,W,24.0,25,58,0.431,10,28,0.357,...,26.666667,10.000000,8.393768,1.843005,4.649721,15.491867,6.954930,9.273017,52.6,44.5


In [26]:
features = Tools.load_pickle('./ml_models/features_for_prediction_new')

mlp_model = load_model('keras_model.h5')

scaler = Tools.load_pickle('keras_scaler')

xgb_model = Tools.load_pickle('./ml_models/XGB')

In [27]:
df = data[features]

assert not(df.isnull().values.any()), 'Nan Values in data'

case = np.isinf(df).values.sum()

assert case is not None, 'Inf Values in data'

In [28]:
X = np.array(df)

y_pred_xgb = xgb_model.predict(X)

In [29]:
mapp = {
    'W': 1,
    'L': 0
}

imap = {
        1: 'W',
        0: 'L'
        }

y_val = np.array(data['W/L_Home'].map(mapp))

In [30]:
print(f'\nf1_score xgb: {Tools.f1_eval(y_pred_xgb, y_val)}\n')
cm = confusion_matrix(y_val, y_pred_xgb)
print(cm)


f1_score xgb: 0.7229894394800975

[[888 342]
 [340 890]]


In [31]:
X = np.array(df)
X = scaler.transform(X)

In [32]:
y_proba_mlp = mlp_model.predict(X)

cond = y_proba_mlp >= 0.5

y_pred_mlp = np.where(cond, 1, 0)

In [33]:
print(f'\nf1_score nn: {Tools.f1_eval(y_pred_mlp, y_val)}\n')
cm = confusion_matrix(y_val, y_pred_mlp)
print(cm)


f1_score nn: 0.7115544472152951

[[910 320]
 [374 856]]


In [13]:
y_proba_mlp

array([[0.6046094 ],
       [0.3513435 ],
       [0.45581114],
       ...,
       [0.24583223],
       [0.7085042 ],
       [0.5672491 ]], dtype=float32)

In [14]:
df = pd.read_csv('validation_data_first_half_2012_2015.csv', index_col=0)

In [15]:
y_val = np.array(df['W/L_Home'].map(mapp))

df = df[features]

assert not(df.isnull().values.any()), 'Nan Values in data'

case = np.isinf(df).values.sum()

assert case is not None, 'Inf Values in data'
df

,PTS_Home,FGM_Home,FGA_Home,FG%_Home,3PM_Home,3PA_Home,3P%_Home,FTM_Home,FTA_Home,OREB_Home,...,eFG%_Home,eFG%_Away,BLK%_Home,BLK%_Away,STL%_Home,STL%_Away,FF_Home,FF_Away,GS_Home,GS_Away
0,48,19,41,46.3,3,10,30.0,7,12,4,...,1.078947,1.138889,9.375000,6.451613,2.278361,4.601001,14.698495,20.773041,35.2,40.8
1,48,18,38,47.4,5,10,50.0,7,10,5,...,1.138889,1.136364,3.571429,14.285714,8.956721,13.185654,19.126548,22.786609,33.7,49.7
2,43,16,34,47.1,3,13,23.1,8,16,6,...,1.093750,1.078947,10.000000,9.523810,11.472100,16.189313,17.035619,19.541687,29.2,39.8
3,54,22,46,47.8,5,13,38.5,5,5,4,...,1.113636,1.025000,11.111111,12.121212,8.074935,10.172526,23.615744,16.343106,44.2,35.9
4,48,19,43,44.2,3,13,23.1,7,10,10,...,1.078947,1.093750,9.523810,10.000000,16.189313,11.472100,19.541687,17.035619,39.8,29.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7373,49,23,51,45.1,0,7,0.0,3,4,3,...,1.000000,1.033333,8.108108,6.818182,11.954858,5.923048,17.595050,19.907970,39.0,17.5
7374,39,15,43,34.9,1,6,16.7,8,12,6,...,1.033333,1.000000,6.818182,8.108108,5.923048,11.954858,19.907970,17.595050,17.5,39.0
7375,47,18,48,37.5,4,15,26.7,7,8,7,...,1.111111,1.166667,9.677419,9.090909,6.424753,8.414109,21.181051,15.387271,34.0,40.9
7376,42,15,40,37.5,3,5,60.0,9,11,5,...,1.100000,1.022727,3.030303,11.428571,9.242828,6.398444,19.689049,21.956902,25.0,39.5


In [16]:
X = np.array(df)
X = scaler.transform(X)



y_proba_mlp = mlp_model.predict(X)

cond = y_proba_mlp >= 0.5

y_pred_mlp = np.where(cond, 1, 0)

In [17]:
print(f'\nf1_score nn: {Tools.f1_eval(y_pred_mlp, y_val)}\n')
cm = confusion_matrix(y_val, y_pred_mlp)
print(cm)


f1_score nn: 0.7488714046175673

[[2528 1161]
 [ 786 2903]]


In [18]:
X = np.array(df)

y_pred_xgb = xgb_model.predict(X)

In [19]:
print(f'\nf1_score xgb: {Tools.f1_eval(y_pred_xgb, y_val)}\n')
cm = confusion_matrix(y_val, y_pred_xgb)
print(cm)


f1_score xgb: 0.7306753458096013

[[2698  991]
 [ 995 2694]]
